# Install dependencies, perform imports, and download the dataset

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
!pip install pandas 
!pip install matplotlib
!pip install torch
!pip install scikit-learn

In [ ]:
%%capture
%cd /home/m4sulaim/jupyter
%cd net_model_dataset

from data_generator import DataGenerator as DataGenerator
from matplotlib import pyplot as plt
import pandas as pd
import ipywidgets as widgets
from utils import *
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'
from IPython.display import display

from vnf_model import *
from slice_model import *
from resource_allocation import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

In [ ]:
ran_data_gen = DataGenerator('./ran/input_dataset.pkl', './ran/output_dataset.pkl', vnf_type='RAN', norm_type='minmax')
ran_model = VNF_Model('ran', 5, [64, 32, 16], 5)
ran_model.to(device)
ran_model.load_weights('./ran/model.pth')

ovs_data_gen = DataGenerator('./ovs/input_dataset.pkl', './ovs/output_dataset.pkl', vnf_type='OvS', norm_type='minmax')
ovs_model = VNF_Model('ovs', 5, [64, 32, 16], 5)
ovs_model.to(device)
ovs_model.load_weights('./ovs/model.pth')

upf_data_gen = DataGenerator('./upf/input_dataset.pkl', './upf/output_dataset.pkl', vnf_type='UPF', norm_type='minmax')
upf_model = VNF_Model('upf', 5, [32, 16], 5)
upf_model.to(device)
upf_model.load_weights('./upf/model.pth')

data_gens = [upf_data_gen, ovs_data_gen, ran_data_gen]
vnf_models = [upf_model, ovs_model, ran_model]
slice_model = Slice_Model(vnf_models, data_gens)


In [ ]:
res_alloc = {'UPF': 200, #CPU (millicores)
                       'OVS': 50, #Throughput (Mbps)
                       'RAN': 1000} #CPU (millicores)
input_throughput = 35

output_throughput = slice_model.predict_throughput(list(res_alloc.values()), input_throughput, res_normalized=False)
print(output_throughput)

In [ ]:
input_throughput = 40
output_throughput = 15
resource_allocation, _, _, _, _ = MicroOpt(slice_model, input_throughput, output_throughput, verbose=1)
print(resource_allocation)

In [ ]:
output_throughput = slice_model.predict_throughput(list(resource_allocation[0]), input_throughput, res_normalized=True)
print(output_throughput)
